### Setup, Installation, and Load Libraries

In [5]:
options(repos = c(CRAN = "https://cloud.r-project.org"))

# Install required packages if missing
required_pkgs <- c("BiocManager", "edgeR", "biomaRt", "amap", "pbapply")
for (pkg in required_pkgs) {
  if (!requireNamespace(pkg, quietly = TRUE)) {
    if (pkg %in% c("edgeR", "biomaRt")) {
      BiocManager::install(pkg, ask = FALSE)
    } else {
      install.packages(pkg)
    }
  }
}
library(amap)
library(edgeR)
library(biomaRt)
library(pbapply)

### Custom implementation of Hobotnica function since the original Hobotnica package is not available on CRAN or Bioconductor
### Hobotnica function definition

In [2]:

Hobotnica <- function(distMatrix, annotation, verbose=FALSE) {
  annotation <- as.vector(annotation)
  
  rank.m <- as.matrix(distMatrix)
  rank.m[lower.tri(rank.m)] <- rank(rank.m[lower.tri(rank.m)])
  rank.m[upper.tri(rank.m)] <- rank(rank.m[upper.tri(rank.m)])
  
  inclass_sum <- 0
  classes <- unique(annotation)
  Ns <- numeric(length(classes))
  
  for (i in seq_along(classes)) {
    class_samples <- which(annotation == classes[i])
    Ns[i] <- length(class_samples)
    inclass_sum <- inclass_sum + sum(rank.m[class_samples, class_samples])
  }
  
  Ns_sum <- sum(Ns)
  biggest_rank <- Ns_sum * (Ns_sum - 1) / 2
  inclass_unique <- sum(Ns * (Ns - 1)) / 2
  maximal_value <- inclass_unique * (2 * biggest_rank - inclass_unique + 1)
  minimal_value <- inclass_unique * (1 + inclass_unique)
  norm_factor <- maximal_value - minimal_value
  
  if (verbose) {
    cat("maximal_value =", maximal_value, "\n")
    cat("minimal_value =", minimal_value, "\n")
    cat("norm_factor =", norm_factor, "\n")
    cat("inclass_sum =", inclass_sum, "\n")
  }
  
  H <- max(0, 1 - (inclass_sum - minimal_value) / norm_factor)
  return(H)
}

### It processes gene identifiers, filters PAM50 genes, computes Kendall distance matrices, calculates the Hobotnica H-score, 
### and estimates the p-value by comparing the real H-score against H-scores from 3000 random gene sets. 
### Note: A custom Hobotnica function is used as the original package is unavailable in CRAN/Bioconductor.


In [4]:

# Set your input files and mode here (adjust paths as needed)
countMatrixFile <- "C:/Users/Ermias/Documents/data visualization/R_project/GSE58135.countmatrix.txt"
annotationFile <- "C:/Users/Ermias/Documents/data visualization/R_project/SraRunTable_GSE58135.pam50.txt"
mode <- "ensembl" 
#args = commandArgs(trailingOnly=TRUE)
#countMatrixFile <- args[1]
#annotationFile <- args[2]
#mode <- args[3]


# PAM50 signature genes
signature <- c('ACTR3B', 'ANLN', 'BAG1', 'BCL2', 'BIRC5', 'BLVRA', 'CCNB1', 'CCNE1', 
               'CDC20', 'CDC6', 'CDH3', 'CENPF', 'CEP55', 'CXXC5', 'EGFR', 'ERBB2', 
               'ESR1', 'EXO1', 'FGFR4', 'FOXA1', 'FOXC1', 'GPR160', 'GRB7', 'KIF2C', 
               'KRT14', 'KRT17', 'KRT5', 'MAPT', 'MDM2', 'MELK', 'MIA', 'MKI67', 
               'MLPH', 'MMP11', 'MYBL2', 'MYC', 'NAT1', 'NDC80', 'NUF2', 'ORC6L', 
               'PGR', 'PHGDH', 'PTTG1', 'RRM2', 'SFRP1', 'SLC39A6', 'TMEM45B', 
               'TYMS', 'UBE2C', 'UBE2T')

print(paste("PAM50 signature length:", length(signature)))

# Convert gene symbols to Ensembl IDs if needed
if (mode == "ensembl") {
  mart <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")
  bm_res <- getBM(values=signature,
                  filters="external_gene_name",
                  mart=mart,
                  attributes=c("external_gene_name", "ensembl_gene_id"))
  signature_ensembl <- unique(bm_res$ensembl_gene_id)
  print(paste("Number of PAM50 Ensembl IDs found:", length(signature_ensembl)))
} else if (mode == "gid") {
  signature_ensembl <- signature
} else {
  stop("Mode should be one of: 'ensembl', 'gid'. Exiting.")
}

# Load count matrix (auto-detect separator)
cm <-read.csv(countMatrixFile, header=TRUE, sep=",", check.names=FALSE, stringsAsFactors=FALSE)
gene_ids <- cm[[1]]
cm <- cm[,-1]
rownames(cm) <- gene_ids
cm_numeric <- as.data.frame(matrix(as.numeric(unlist(cm)), nrow = nrow(cm), ncol = ncol(cm)))
colnames(cm_numeric) <- colnames(cm)
rownames(cm_numeric) <- gene_ids
cm <- cm_numeric
print(paste("Count matrix dimensions:", dim(cm)[1], "genes x", dim(cm)[2], "samples"))


# Load annotation file
anno_raw <- read.csv(annotationFile, header=TRUE, sep=",", check.names=FALSE, stringsAsFactors=FALSE)
colnames(anno_raw)[1:2] <- c("Run", "source_name")
anno <- data.frame(group = anno_raw$source_name, row.names=anno_raw$Run)

# Find shared samples between count matrix and annotation
shared_samples <- intersect(colnames(cm), rownames(anno))
if (length(shared_samples) == 0) {
  stop("No shared samples between count matrix and annotation file!")
}
print(paste("Number of shared samples:", length(shared_samples)))

# Subset count matrix and annotation to shared samples
cm <- cm[, shared_samples]
anno <- anno[shared_samples, , drop=FALSE]

# Filter genes by PAM50 signature
set_filt <- intersect(rownames(cm), signature_ensembl)
print(paste("Number of PAM50 genes found in count matrix:", length(set_filt)))
if (length(set_filt) < 10) {
  warning("Less than 10 PAM50 genes found - results may be unreliable.")
}

# Generate random signatures
randomSignaturesList <- vector("list", 3000)
all_genes <- rownames(cm)
sig_length <- length(set_filt)

if (length(all_genes) < sig_length) {
  stop(paste("Insufficient genes for random sampling. Available:", length(all_genes), "Needed:", sig_length))
}

for (i in 1:3000) {
  randomSignaturesList[[i]] <- sample(all_genes, sig_length)
}

# Calculate Hobotnica scores for random signatures
randomSigScores <- pbsapply(randomSignaturesList, function(gset) {
  submat <- cm[gset, , drop=FALSE]
  distMatrix <- as.matrix(Dist(t(submat), method="kendall", nbproc=10))
  Hobotnica(distMatrix, anno$group)
})

# Calculate Hobotnica score for PAM50 genes
cm_subset <- cm[set_filt, , drop=FALSE]
distMatrix <- as.matrix(Dist(t(cm_subset), method="kendall", nbproc=10))
score <- Hobotnica(distMatrix, anno$group)

# Calculate p-value
p_value <- min(1, (sum(randomSigScores >= score) + 1) / (length(randomSigScores) + 1))

# Print results
cat("Hobotnica H-score: ", score, "\n")
cat("P-value: ", p_value, "\n")


[1] "PAM50 signature length: 50"
[1] "Number of PAM50 Ensembl IDs found: 52"
[1] "Count matrix dimensions: 60676 genes x 169 samples"
[1] "Number of shared samples: 140"
[1] "Number of PAM50 genes found in count matrix: 49"
Hobotnica H-score:  0.8351978 
P-value:  0.0003332223 
